In [155]:
import numpy as np
from matplotlib import pyplot as plt
from sionna.fec.turbo import TurboEncoder, TurboDecoder
from PIL import Image
import io

In [156]:
def encode_string(s):
    # Convert the string to a list of ASCII values
    ascii_values = [ord(c) for c in s]

    # Convert each ASCII value to an 8-bit binary representation
    bit_arrays = [np.array(list(format(val, '08b')), dtype=np.int32) for val in ascii_values]

    # Flatten the list of bit arrays into a single array
    bit_array = np.concatenate(bit_arrays)

    # Reshape array to have a single row
    bit_array = bit_array.reshape(1, -1)

    return bit_array

def decode_string(bit_array):
    # Ensure the array is 1-dimensional
    bit_array = np.array(bit_array.flatten(), dtype=int)

    # Split the bit array into chunks of 8 bits
    n = 8
    byte_chunks = [bit_array[i:i + n] for i in range(0, len(bit_array), n)]

    # Convert each chunk of 8 bits to an ASCII value
    ascii_values = [int(''.join(map(str, chunk)), 2) for chunk in byte_chunks]

    # Convert the ASCII values to characters and join them into a string
    string = ''.join(chr(val) for val in ascii_values)

    return string


In [157]:
def encode_image_to_binary_array(image_path):
    # Bild öffnen und in Schwarz-Weiß konvertieren
    with Image.open(image_path) as img:
        img = img.convert('1')  # '1' steht für 1-bit Pixel, schwarz und weiß

    # Bild in NumPy-Array konvertieren
    img_array = np.array(img)

    # Array in 1D umformen und in int32 konvertieren
    binary_array = img_array.flatten().astype(np.int32)

    # In das gewünschte Format [[databits]] umwandeln
    return np.array([binary_array])


def decode_binary_array_to_image(binary_array, original_shape):
    # Array zurück in 2D-Form bringen
    img_array = binary_array.reshape(original_shape)

    # NumPy-Array in ein Bild konvertieren
    img = Image.fromarray((img_array * 255).astype(np.uint8))

    # Bild in Bytes-Objekt umwandeln
    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='PNG')

    return img_byte_arr.getvalue()

In [158]:
def split_into_fragments(u, frame_length=6144):
    # Berechne die Anzahl der Fragmente
    num_fragments = int(np.ceil(u.size / frame_length))

    # Erstelle eine Liste, um die Fragmente zu speichern
    fragments = []

    # Teile das Array u in Fragmente der Größe frame_length
    for i in range(num_fragments):
        start_idx = i * frame_length
        end_idx = min((i + 1) * frame_length, u.size)
        fragment = u[0, start_idx:end_idx]
        fragments.append(fragment)

    # Konvertiere die Liste der Fragmente in ein NumPy-Array
    fragments = np.array(fragments)

    return fragments

In [159]:
encoder = TurboEncoder(constraint_length=4, # Desired constraint length of the polynomials
                       rate=1/3,  # Desired rate of Turbo code
                       terminate=True) # Terminate the constituent convolutional encoders to all-zero state


# the decoder can be initialized with a reference to the encoder
decoder = TurboDecoder(encoder,
                       num_iter=6, # Number of iterations between component BCJR decoders
                       algorithm="map", # can be also "maxlog"
                       hard_out=True) # hard_decide output

# Encode Data

## Text Encoding

In [160]:
# Originale Informationsbits
#u = np.array([[1, 0, 1, 0]], dtype=np.int32)
string_message = "Hallo!"
u = encode_string(string_message)
print("Originale Bits:", u)

Originale Bits: [[0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 1 1 0 0 0 1 1 0 1 1 0 0 0 1 1 0
  1 1 1 1 0 0 1 0 0 0 0 1]]


# Channel Coding

In [161]:
# Kodierung
c = encoder(u)
encoded = c.numpy()
print("Kodierte Bits:    ", encoded)

Kodierte Bits:     [[0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1.
  0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0.
  0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0.
  0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1.
  0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1.
  1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0.]]


In [162]:
# Simulation eines Bitfehlers
llr = encoded.copy()
error_positions = np.array([5,10])
error_num = 1
error_positions = np.random.choice(range(llr[0].size-1), size=error_num, replace=False)
llr[0, error_positions] = np.where(llr[0, error_positions] == 0, 1.0, 0.0)
print(error_positions)
print("LLR mit Bitfehler:", llr)

[62]
LLR mit Bitfehler: [[0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1.
  0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0.
  0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0.
  0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1.
  0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1.
  1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0.]]


In [163]:
# Dekodierung
u_hat = decoder(llr)
decoded = u_hat.numpy()
print("Dekodierte Bits:", decoded)

# Überprüfung der Fehlerkorrektur
if np.array_equal(u, decoded):
    print("Fehler erfolgreich korrigiert!")
    print("Dekodierte Nachricht: " + decode_string(decoded))
else:
    print("Fehlerkorrektur fehlgeschlagen.")
    print("Dekodierte Nachricht: " + decode_string(decoded))

Dekodierte Bits: [[0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0.
  0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1.]]
Fehler erfolgreich korrigiert!
Dekodierte Nachricht: Hallo!


In [ ]:
## Ergebnisse Plotten
# Simulierte Daten
u_plot = u[0]
encoded_plot = encoded[0]
llr_plot = llr[0]
decoded_plot = decoded[0]
### Turbocodierte Bits

In [ ]:
# Plot für kodierte Bits und LLR-Werte
plt.figure(figsize=(12, 4))
plt.stem(range(len(encoded_plot)), encoded_plot, linefmt='r-', markerfmt='ro', basefmt=" ", label="Kodierte Bits")
plt.stem(range(len(llr_plot)), llr_plot, linefmt='b-', markerfmt='bo', basefmt=" ", label="Bits mit Fehler (LLR)")
plt.title("Kodierte Bits und LLR-Werte")
plt.xlabel("Bit-Index")
plt.ylabel("Wert")
plt.xticks(range(len(encoded_plot)))
plt.yticks([0,1])
plt.grid(True)
plt.legend()
plt.show()


### Plot für originale und dekodierte Bits

In [ ]:
# Erstellen der Subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Originale Bits Plot
ax1.stem(range(len(u_plot)), u_plot, linefmt='b', markerfmt='bo', basefmt=" ", label="Originale Bits")
ax1.set_title("Originale Bits")
ax1.set_xlabel("Bit-Index")
ax1.set_ylabel("Bit-Wert")
ax1.set_xticks(range(len(u_plot)))
ax1.set_yticks([0, 1])
ax1.grid(True)
ax1.legend()

# Dekodierte Bits Plot
ax2.stem(range(len(decoded_plot)), decoded_plot, linefmt='r', markerfmt='ro', basefmt=" ", label="Dekodierte Bits")
ax2.set_title("Dekodierte Bits")
ax2.set_xlabel("Bit-Index")
ax2.set_ylabel("Bit-Wert")
ax2.set_xticks(range(len(decoded_plot)))
ax2.set_yticks([0, 1])
ax2.grid(True)
ax2.legend()

# Layout anpassen und anzeigen
plt.tight_layout()
plt.show()

In [ ]:
print(np.array(decoded[0], dtype=int))
decoded = np.array(decoded[0], dtype=int)
decode_string(decoded)

# Image Example

In [ ]:
image_path = "../data/images/image.png"
u = encode_image_to_binary_array(image_path)
print("Originale Bits:", u)

In [ ]:
information_codeword_length = u[0].size
fragments = split_into_fragments(u)
print(len(fragments))